In [61]:
import numpy as np
import pandas as pd
import csv
import itertools
import time

In [68]:
# save data
def save_data_tocsv(data, path, filename, encoding):
    data.to_csv(path + filename, index=False, encoding = encoding)
    
# load data
def _load_csvdata(path, filename, encoding): # encoding 如不需要可寫 None。
    data = pd.read_csv(path + filename, encoding = encoding)
    return data


In [63]:
### 基礎設定
# 設定要看幾個重要變數
num = 20 
model_dict = {'HOS':'住院險', 'SUR':'手術險', 'REI':'實支實付', 'ACC':'意外險', 'DD':'重疾險', 'LTC':'長照險', 'LIFE':'壽險'}
time = '2023_01_03'

In [64]:
var_path = 'D:\LEE\BOX_ALL\BOX_NEW\行銷分ABT與模型檔\正式模型區\保障型行銷分\data\importance' + '\\'

In [65]:
var_list = pd.read_csv(var_path+"變數紀錄表_0905.csv", encoding='cp950')[['變數名稱', '中文名稱']]
var_list.head()

,變數名稱,中文名稱
0,BMI,BMI
1,TC_APC_INSD_IND,曾為心安逸要或被保人
2,L1YR_CREDIT_IND,近一年有申辦信用卡
3,L1YR_ACCOUNT_IND,近一年有開戶
4,L1YR_MORTGAGE_LOAN_IND,近一年有申辦放款


In [66]:
myindex = list(range(num))
var_top20 = pd.DataFrame(myindex, columns=['index'],index=myindex)
for i in model_dict:
    # 平均5個fold的importance 並排序
    var = _load_csvdata(var_path, time+i+'_重要特徵.csv', None)
    var = var[var['fold'] == 5]
    var[model_dict[i]+'平均importance'] = var['cumsum_2']/5
    var[model_dict[i]+'rank'] = var[model_dict[i]+'平均importance'].rank(ascending=False)
    var['變數名稱'] = var['feature']
    var = var[['變數名稱', model_dict[i]+'平均importance', model_dict[i]+'rank']]
    # 串中文名稱
    var_tmp = var_list.merge(var, how = 'inner')
    # 統整7個險種之重要變數
    imp_list = var_tmp[var_tmp[model_dict[i]+'rank'] <= 20].sort_values(by=model_dict[i]+'rank',ascending=True)['中文名稱'].tolist()
    imp_df = pd.DataFrame(imp_list, columns=[model_dict[i]], index=list(range(len(imp_list)))).reset_index()
    var_top20 = var_top20.merge(imp_df, on='index', how = 'left')
#檢視迴圈結果
var_top20

,index,住院險,手術險,實支實付,意外險,重疾險,長照險,壽險
0,0,最近一次透過業務員投保間隔時間,最近一次透過業務員投保間隔時間,家戶理賠醫療日額(燒燙傷)距今天數,家戶理賠醫療日額(燒燙傷)距今天數,最近一次透過業務員投保間隔時間,最近一次透過業務員投保間隔時間,最近一次透過業務員投保間隔時間
1,1,保險年齡,保險年齡,保險年齡,最近一次成為被保人距今間隔時間(日),最近一次成為要保人距今間隔時間(日),最近一次成為被保人距今間隔時間(日),與在職經手業務員購買次數(取最高的業務員)
2,2,法定年齡,法定年齡,最近一次透過業務員投保間隔時間,保險年齡,最近一次成為被保人距今間隔時間(日),保險年齡,最近一次成為被保人距今間隔時間(日)
3,3,最近一次成為要保人距今間隔時間(日),家戶理賠醫療日額(燒燙傷)距今天數,與在職經手業務員購買次數(取最高的業務員),法定年齡,與在職經手業務員購買次數(取最高的業務員),BMI,家戶理賠醫療日額(燒燙傷)距今天數
4,4,家戶理賠醫療日額(燒燙傷)距今天數,最近一次成為要保人距今間隔時間(日),法定年齡,最近一次成為要保人距今間隔時間(日),家戶理賠醫療日額(燒燙傷)距今天數,家戶理賠醫療日額(燒燙傷)距今天數,教育程度/學歷
5,5,與在職經手業務員購買次數(取最高的業務員),與在職經手業務員購買次數(取最高的業務員),最近一次成為被保人距今間隔時間(日),最近一次透過業務員投保間隔時間,近5年投保主約件數,法定年齡,近5年投保主約件數
6,6,客戶標籤_車險持有張數,最近一次成為被保人距今間隔時間(日),BMI,近一個月步數達標次數,教育程度/學歷,最近一次成為要保人距今間隔時間(日),家戶理賠醫療實支距今天數
7,7,近5年投保主約件數,近一年被拜訪次數,家戶理賠醫療實支距今天數,家戶理賠醫療實支距今天數,近一年被拜訪次數,與在職經手業務員購買次數(取最高的業務員),近一年被拜訪次數
8,8,最近一次成為被保人距今間隔時間(日),近一個月步數達標次數,近一個月步數達標次數,近一年被拜訪次數,近一個月步數達標次數,教育程度/學歷,近一個月步數達標次數
9,9,家戶理賠醫療實支距今天數,客戶標籤_車險持有張數,以被保人身分過去持有之主約張數總和,與在職經手業務員購買次數(取最高的業務員),近5年投保主約件數(等級),近一個月步數達標次數,最近一次成為要保人距今間隔時間(日)


In [69]:
save_data_tocsv(var_top20, var_path, '各模型前20名變數.csv', 'cp950')